# Impact COVID-19 in The Netherlands

Plotting diseased per week of the years 2020- on boxplots.

The graphs are shown:
- normalized (# people per week / (# weekdays * estimated population in that week))
- per week
- per category (total, total men, total women, age 0-65, age 65-80, age 80 plus)
- line graphs (2020 - present)
- box plots (1995 or later (user choice) - 2019)

<br>
All the data used is from CBS (Dutch sensus bureau). Running the notebook updates the data.

<br> 
** note
- Diseased per category per week: [CBS table 70895NED](https://opendata.cbs.nl/statline/#/CBS/nl/dataset/70895NED)
- Extrapolation for recent week(s) for diseased categories that still have n/a  
- Population per year: [CBS table 70072NED](https://opendata.cbs.nl/statline/#/CBS/nl/dataset/70072NED)
- Interpolation is used to estimate population per week between two years
- Estimates is used for population next year: [CBS table 84527NED](https://opendata.cbs.nl/statline/#/CBS/nl/dataset/84527NED)



In [1]:
# Most important constants that can be manually tweaked; RESTART AND RUN ALL the complete notebook again!!

CUTOFF = 2020   # year in which COVID-19 came into existance
EXTRAPOLATION_FROM = 2015 # year to extrapolate n/a values for age categories <== change this value if you find it appropiate to use another year


## Setting things up

In [2]:
# installing the necessary libraries for this notebook
import cbsodata
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import plotly.graph_objects as go
from ipywidgets import interact

# Defining the required global constants
CURRENT_YEAR = datetime.now().year
NEXT_YEAR = datetime.now().year + 1

# CBS Open Data Tables
DISEASED = '70895ned'
POPULATION = '70072NED'
EST_POPULATION = '84527NED'

## Getting and prepping CBS data

In [3]:
# get 70895NED CBS data: overledenen per week
data1_1 = pd.DataFrame(cbsodata.get_data(DISEASED))

In [4]:
# remove all non Jaar-Week. Make a copy for .map(len) > 5 creates a slice and this doesn't work nicely with later operations
data1_2 = data1_1[data1_1['Perioden'].map(len) > 5].copy()

# # calculate deaths per week
data1_2['dagen'] = [7 if x.find('(') == -1 else int(x[x.find('(')+1:x.find('(')+2]) for x in data1_2['Perioden']]
data1_2['Overledenen_2'] = data1_2['Overledenen_1'] / data1_2['dagen'] #[7 if x.find('(') == -1 else int(x[x.find('(')+1:x.find('(')+2]) for x in data1_1['Perioden']]

# create a pivot table
data1_3 = pd.pivot_table(data1_2, values='Overledenen_2', index=['Perioden'], columns=['Geslacht', 'LeeftijdOp31December'])

In [5]:
# flatten the pivot table
data1_3.columns = data1_3.columns.to_series().str.join('-')
data1_4 = data1_3.reset_index(level=0)

# find out # of days for the record and remove the (x dag)
data1_4['Perioden1'] = [x if x.find('(') == -1 else x[0:x.find('(')-1] for x in data1_4['Perioden']]
data1_4['jaar'] = [int(x[0:5]) for x in data1_4['Perioden1']]
data1_4['week'] = [int(x[10:]) for x in data1_4['Perioden1']]

data1_4 = data1_4.rename(columns={'Totaal mannen en vrouwen-Totaal leeftijd': 'k_totaal',
                            'Mannen-Totaal leeftijd': 'k_mannen',
                            'Vrouwen-Totaal leeftijd': 'k_vrouwen'
                            })

In [6]:
# grouping the data for CBS data does not provide data per age category number of men and woman
data1_4['k_0_65'] = data1_4['Mannen-0 tot 65 jaar'] + data1_4['Vrouwen-0 tot 65 jaar']
data1_4['k_65_80'] = data1_4['Mannen-65 tot 80 jaar'] + data1_4['Vrouwen-65 tot 80 jaar']
data1_4['k_80plus'] = data1_4['Mannen-80 jaar of ouder'] + data1_4['Vrouwen-80 jaar of ouder']

columns_list = ['k_totaal', 'k_mannen', 'k_vrouwen', 'k_0_65', 'k_65_80', 'k_80plus']
for item in columns_list:
  data1_4[item] = data1_4[item].round(2)


data1_4 = data1_4[['jaar', 'week', 'k_totaal', 'k_mannen', 'k_vrouwen', 'k_0_65', 'k_65_80', 'k_80plus']]
df_overledenen = data1_4.sort_values(['jaar', 'week'])

# print(df_overledenen)

In [7]:
# getting the data from 70072NED, this data set contains population over time per year

filtering = "substringof('NL',RegioS)"
selection = ['Perioden', 'TotaleBevolking_1', 'Mannen_2', 'Vrouwen_3', 'k_65Tot80Jaar_11', 'k_80JaarOfOuder_12']
data2_1 = pd.DataFrame(cbsodata.get_data(POPULATION,
                                      filters=filtering,
                                      select=selection
                                      ))
data2_1 = data2_1.rename(columns={'Perioden': 'jaar',
                            'TotaleBevolking_1': 'k_totaal_t',
                            'Mannen_2': 'k_mannen_t',
                            'Vrouwen_3': 'k_vrouwen_t',
                            'k_65Tot80Jaar_11': 'k_65_80_t',
                            'k_80JaarOfOuder_12': 'k_80plus_t'})
data2_1['k_0_65_t'] = data2_1['k_totaal_t'] - data2_1['k_65_80_t'] - data2_1['k_80plus_t']
data2_1 = data2_1[['jaar', 'k_totaal_t', 'k_mannen_t', 'k_vrouwen_t', 'k_0_65_t', 'k_65_80_t', 'k_80plus_t']]

df_bevolkings_aantal = data2_1.copy()
df_bevolkings_aantal['jaar'] = df_bevolkings_aantal['jaar'].astype(int)

In [8]:
# CBS does not provide details per category of the lastest weeks, extrapolation is used to fill in the n/a
df=df_bevolkings_aantal

df=df[df['jaar']>EXTRAPOLATION_FROM]

correction = {
    'k_mannen_t': (df['k_mannen_t'] / df['k_totaal_t']).mean(),
    'k_vrouwen_t': (df['k_vrouwen_t'] / df['k_totaal_t']).mean(),
    'k_0_65_t': (df['k_0_65_t'] / df['k_totaal_t']).mean(),
    'k_65_80_t': (df['k_65_80_t'] / df['k_totaal_t']).mean(),
    'k_80plus_t': (df['k_80plus_t'] / df['k_totaal_t']).mean()
    }

## Check if EXTRAPOLATION_FROM is correct

Play with the slider to see from which year you think the MEAN should be used to extrapolate the N/A values that might be present in the latest CBS data set for diseased.

If you want to use another YEAR, update EXTRAPOLATION_FROM in the first cell of this notebook and rerun this notebook in full.

In [9]:
# CBS does not provide details per category of the lastest weeks, extrapolation is used to fill in the n/a

print(f'''Jear {EXTRAPOLATION_FROM} is used to estimated the n/a values in the CBS data of the most recent week(s).

If you like to use a different date BY DEFAULT, update this in the first cell EXTRAPOLATION_FROM and then the Notebook requires RESTART AND RERUN ALL.

''')

@interact(jaar=(min(df_bevolkings_aantal['jaar']), CUTOFF, 1))
def update(jaar=EXTRAPOLATION_FROM):
  df_temp = df_bevolkings_aantal.copy()
  df_temp=df_temp[df_temp['jaar']>jaar]
  df_temp['p_mannen'] = df_temp['k_mannen_t'] / df_temp['k_totaal_t']
  df_temp['p_vrouwen'] = df_temp['k_vrouwen_t'] / df_temp['k_totaal_t']
  df_temp['p_0_65'] = df_temp['k_0_65_t'] / df_temp['k_totaal_t']
  df_temp['p_65_80'] = df_temp['k_65_80_t'] / df_temp['k_totaal_t']
  df_temp['p_80plus'] = df_temp['k_80plus_t'] / df_temp['k_totaal_t']
  
  print(df_temp[['p_mannen', 'p_vrouwen', 'p_0_65', 'p_65_80', 'p_80plus']].describe())


Jear 2015 is used to estimated the n/a values in the CBS data of the most recent week(s).

If you like to use a different date BY DEFAULT, update this in the first cell EXTRAPOLATION_FROM and then the Notebook requires RESTART AND RERUN ALL.




interactive(children=(IntSlider(value=2015, description='jaar', max=2020, min=1995), Output()), _dom_classes=(…

## Continuing to get and prep CBS data

In [10]:
# getting the data prognose next year

filtering = "((Perioden eq '"+str(NEXT_YEAR)+"JJ00')) and ((Leeftijd eq '10000')) and ((PrognoseInterval eq 'MW00000')) and ((substringof('NL',RegioIndeling2018)))"
selection = ['Perioden', 'TotaleBevolking_1']
data = pd.DataFrame(cbsodata.get_data(EST_POPULATION,
                                      select=selection,
                                      filters=filtering
                                      ))
# Rename Bevolking to match df_70072NED
data = data.rename(columns={'Perioden': 'jaar'})
# data['jaar'] = pd.to_numeric(data['jaar'])

data['k_totaal_t'] = data['TotaleBevolking_1'] * 1000
data.drop('TotaleBevolking_1', axis=1, inplace=True)

prognose_nextyear = data.copy()

df_bevolkings_aantal = pd.concat([df_bevolkings_aantal, prognose_nextyear])
df_bevolkings_aantal['jaar'] = df_bevolkings_aantal['jaar'].apply(pd.to_numeric)
df_bevolkings_aantal.reset_index(drop=True, inplace=True)

In [11]:
df_bevolkings_aantal['week'] = 1
df_bevolkings_aantal = df_bevolkings_aantal[['jaar', 'week', 'k_totaal_t', 'k_mannen_t', 'k_vrouwen_t', 'k_0_65_t', 'k_65_80_t', 'k_80plus_t']]

for year in [int(CURRENT_YEAR), int(NEXT_YEAR)]:
  for item in columns_list[1:]:
    bevolking = df_bevolkings_aantal.loc[df_bevolkings_aantal['jaar']==year, 'k_totaal_t']
    df_bevolkings_aantal.loc[df_bevolkings_aantal['jaar']== year, item+'_t'] = correction[item+'_t']*bevolking

In [12]:
# temp keeps track which rows must be deleted after
temp = df_overledenen['jaar'].count()
df_temp = df_overledenen
# Add all weeks of the current year not yet in the dataframe
latest_week = df_temp['week'].iloc[-1]
latest_year = df_temp['jaar'].iloc[-1]
for i in range(latest_week + 1, 53):
    add_row = {'jaar': latest_year, 'week': i}
    # df_temp = df_temp.append(add_row, ignore_index=True)
    df_temp_row = pd.DataFrame(add_row, index=[0])
    pd.concat([df_temp, df_temp_row])
# add first week of next year to get the predicted population, needed for interpolation of current year
# df_temp = df_temp.append({'jaar': latest_year+1, 'week': 1}, ignore_index=True)
df_temp_row = pd.DataFrame({'jaar': latest_year+1, 'week': 1}, index=[0])
pd.concat([df_temp, df_temp_row])

df_temp = pd.merge(df_temp, df_bevolkings_aantal, how='left', left_on=['jaar', 'week'], right_on=['jaar', 'week'])

# Interpolate for all other weeks
for item in columns_list:
  df_temp[item+'_t'] = df_temp[item+'_t'].interpolate().astype(float)

# Remove added Jaar and Week, drop first row (is full of nans) and reindex
df_temp.drop(df_temp.index[temp:], inplace=True)
df_temp = df_temp.iloc[1:,:]

df_temp.reset_index(drop=True, inplace=True)
df_temp['jaar'] = df_temp['jaar'].astype(int)
df_overledenen2 = df_temp.copy()


In [13]:
# normalize
df_temp = df_overledenen2.copy()
for item in columns_list:
  df_temp[item] = (df_temp[item] / df_temp[item+'_t'] * 100_000).round(2)
  df_temp.drop(item+'_t', axis=1, inplace=True)
df_overledenen3 = df_temp
# print(df_overledenen3)

## Display the graphs in an interactive plots

In [14]:
# generic lists
colors = ['red', 'blue', 'green', 'purple', 'orange']
plots = ['k_totaal', 'k_mannen', 'k_vrouwen', 'k_0_65', 'k_65_80', 'k_80plus']

weeks = list(df_overledenen3['week'])
years = list(set(df_overledenen3['jaar']))

In [15]:
df_temp = df_overledenen.copy()
df_temp['week']=1
df_temp = df_temp.groupby(by='jaar').sum()
for item in df_temp.columns[1:]:
  df_temp[item] = (df_temp[item] / df_temp['week']).round(1)
df_temp.drop('week', inplace=True, axis=1)
df_temp.reset_index(inplace=True)

### Impact COVID-19 in The Netherlands

Showing per category the diseased per year from 2020 compared to boxplot of that category 1995 (or later via slider) through 2019.

In [16]:
print('Select the category to see another category')
df_temp1 = df_overledenen.copy()
df_bar = df_temp1.groupby('jaar').mean()

title = f'Diseased in The Netherlands<br><sup>CBS OpenData tables {DISEASED}, {POPULATION} and {EST_POPULATION}, updated on: ' + datetime.today().strftime('%Y-%m-%d') 

fig3 = go.FigureWidget()
fig3.layout.title = title
fig3.layout.width = 750
fig3.layout.height = 400
fig3.layout.xaxis.title = 'years'
fig3.layout.xaxis.tickvals = years
fig3.layout.yaxis.title = 'Normalized Average Diseased / 100.000 per day<br><sub>avg((diseased per week / (# days per week * population))</sub>'


fig3.add_bar()

@interact(category=plots)
def update(category='k_totaal'):
  fig3.layout.title = title + f'<br>Data for {category[2:].replace("_", " - ").upper()} from {min(years)} through {NEXT_YEAR-1}'
  with fig3.batch_update():
    fig3.data[0].x = years
    fig3.data[0].y = df_bar[category]
    fig3.data[0].name = category[2:].replace('_', ' - ')
    fig3.layout.yaxis.range = [df_bar[category].min()-50,df_bar[category].max()+50]
  fig3.show()


Select the category to see another category


interactive(children=(Dropdown(description='category', options=('k_totaal', 'k_mannen', 'k_vrouwen', 'k_0_65',…

In [17]:
print(f'Select the YEAR to adjust the start of the boxplot data. Data for the boxplotes ends {CUTOFF}\n')
title = f'Impact COVID-19 in The Netherlands<br><sup>CBS OpenData tables {DISEASED}, {POPULATION} and {EST_POPULATION}, updated on: ' + datetime.today().strftime('%Y-%m-%d') 

fig2 = go.FigureWidget()
fig2.layout.title = title
fig2.layout.width = 900
fig2.layout.height = 600
fig2.layout.xaxis.title = 'Categories'
# fig2.layout.xaxis.tickvals = weeks
fig2.layout.yaxis.title = 'Normalized Diseased per day / 100.000<br><sub>(diseased per jear / (# days per year * population)</sub>'
cat='k_totaal'

for plot in df_temp.columns[1:]:
  fig2.add_box()
  for j in range(CUTOFF, NEXT_YEAR):
    fig2.add_scatter()

@interact(year=(min(df_temp['jaar']), CUTOFF-1, 1))
def update(year=EXTRAPOLATION_FROM):
  fig2.layout.title = title + f'<br>Boxplot data from {year} through {CUTOFF-1}'
  with fig2.batch_update():
    # fig2.data[0].x = cat
    for i, plot in enumerate(df_temp.columns[1:]):
      # fig2.data[i*(NEXT_YEAR-CUTOFF+1)].x = [plot]
      fig2.data[i*(NEXT_YEAR-CUTOFF+1)].y = df_temp[(df_temp['jaar']>=year)&(df_temp['jaar']<CUTOFF)][plot]
      fig2.data[i*(NEXT_YEAR-CUTOFF+1)].name = plot[2:].replace('_', ' tot ')
      for j in range(CUTOFF, NEXT_YEAR):
        fig2.data[i*(NEXT_YEAR-CUTOFF+1)+(j-CUTOFF+1)].x = [plot[2:].replace('_', ' tot ')]
        fig2.data[i*(NEXT_YEAR-CUTOFF+1)+(j-CUTOFF+1)].y = df_temp[df_temp['jaar']==j][plot]
        fig2.data[i*(NEXT_YEAR-CUTOFF+1)+(j-CUTOFF+1)].name = j
        fig2.data[i*(NEXT_YEAR-CUTOFF+1)+(j-CUTOFF+1)].line.color = colors[j-CUTOFF]
        if (i>0):
          fig2.data[i*(NEXT_YEAR-CUTOFF+1)+(j-CUTOFF+1)].showlegend = False
        else:
          fig2.data[i*(NEXT_YEAR-CUTOFF+1)+(j-CUTOFF+1)].showlegend = True
  fig2.show()

Select the YEAR to adjust the start of the boxplot data. Data for the boxplotes ends 2020



interactive(children=(IntSlider(value=2015, description='year', max=2019, min=1995), Output()), _dom_classes=(…

### Impact COVID-19 in The Netherlands

Showing per user selected category how the diseased of the years 2020-today relate to boxplot data from 1995 (user can choose later year) through 2019.

In [18]:
print('Select the category to see another category')
print(f'Select the YEAR to adjust the start of the boxplot data. Data for the boxplotes ends {CUTOFF}\n')
df = df_overledenen3.copy()

title = f'Impact COVID-19 in The Netherlands<br><sup>CBS OpenData tables {DISEASED}, {POPULATION} and {EST_POPULATION}, updated on: ' + datetime.today().strftime('%Y-%m-%d') 

fig1 = go.FigureWidget()
fig1.layout.title = title
fig1.layout.width = 1200
fig1.layout.height = 600
fig1.layout.xaxis.title = 'week'
fig1.layout.xaxis.tickvals = weeks
fig1.layout.yaxis.title = 'Normalized Diseased / 100.000<br><sub>(diseased per week / (# days per week * population)</sub>'

fig1.add_box()
for i in range(CUTOFF, NEXT_YEAR):
  fig1.add_scatter()

@interact(category=plots, year=(min(df['jaar']), CUTOFF-1, 1))
def update(category='k_totaal', year=EXTRAPOLATION_FROM):
  fig1.layout.title = title + f'<br>Boxplot data for {category[2:].replace("_", " - ").upper()} from {year} through {CUTOFF-1}'
  with fig1.batch_update():
    fig1.data[0].x = weeks
    fig1.data[0].y = df[(df['jaar']>=year)&(df['jaar']<CUTOFF)][category]
    fig1.data[0].name = category[2:].replace('_', ' - ')
    for i in range(CUTOFF, NEXT_YEAR):
      fig1.data[i+1-CUTOFF].line.color=colors[i-CUTOFF]
      fig1.data[i+1-CUTOFF].x = df[df['jaar']==i]['week']
      fig1.data[i+1-CUTOFF].y = df[df['jaar']==i][category]
      fig1.data[i+1-CUTOFF].name = i
  fig1.show()

Select the category to see another category
Select the YEAR to adjust the start of the boxplot data. Data for the boxplotes ends 2020



interactive(children=(Dropdown(description='category', options=('k_totaal', 'k_mannen', 'k_vrouwen', 'k_0_65',…